<a href="https://colab.research.google.com/github/MFuzikawa/Imers-o_IA_Criando_Agentes/blob/main/imers_o_ia_alura_google_gemini_aula_05_agentes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip -q install google-genai

In [ ]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [ ]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [ ]:
# Pergunta ao Gemini uma informação mais recente que seu conhecimento

from IPython.display import HTML, Markdown

# Perguntar pro modelo quando é a próxima imersão de IA ###############################################
response = client.models.generate_content(
    model = MODEL_ID,
    contents = 'Quando é a proxima imersão da alura'
)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {response.text}"))

Resposta:
 A Alura realiza imersões com frequência, geralmente com foco em diferentes áreas da tecnologia. Para saber quando será a próxima imersão e qual o tema, a melhor forma é verificar os canais oficiais da Alura:

*   **Site da Alura:** Procure na página principal ou na seção de eventos/cursos.
*   **Redes sociais da Alura:** Fique de olho nos perfis da Alura no LinkedIn, Instagram, Twitter e Facebook.
*   **Comunidade da Alura no Discord:** Lá você pode encontrar informações sobre os próximos eventos e interagir com outros alunos.
*   **Blog da Alura:** Eles costumam anunciar as imersões no blog.
*   **E-mail:** Se você é assinante da Alura, verifique sua caixa de entrada, pois eles costumam enviar informações sobre os próximos eventos por e-mail.

Ao verificar esses canais, você terá acesso às informações mais atualizadas sobre a próxima imersão da Alura.

In [ ]:
# Pergunta ao Gemini uma informação utilizando a busca do Google como contexto

response = client.models.generate_content(
    model=MODEL_ID,
    contents='Quando é a próxima Imersão IA com Google Gemini da Alura?',
    config = {"tools":[{"google_search": {}}]}
    # Inserir a tool de busca do Google ###############################################
)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {response.text}"))

Resposta:
 A próxima Imersão IA com Google Gemini da Alura acontecerá entre os dias 12 e 16 de maio de 2025. As inscrições estão abertas até o dia 11 de maio de 2025.


In [ ]:
# Exibe a busca
print(f"Busca realizada: {response.candidates[0].grounding_metadata.web_search_queries}")
# Exibe as URLs nas quais ele se baseou
print(f"Páginas utilizadas na resposta: {', '.join([site.web.title for site in response.candidates[0].grounding_metadata.grounding_chunks])}")
print()
display(HTML(response.candidates[0].grounding_metadata.search_entry_point.rendered_content))

Busca realizada: ['Alura Imersão IA com Google Gemini data']
Páginas utilizadas na resposta: alura.com.br, tecmundo.com.br



In [ ]:
# Instalar Framework ADK de agentes do Google ################################################
!pip install -q google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.

In [ ]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [ ]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [ ]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
##########################################
# --- Agente 1: Buscador de Notícias --- #
##########################################
def agente_buscador(topico, dia_de_hoje):
    buscador = Agent(
        name="agente_buscador",
        model="gemini-2.0-flash",
        description="Agente que busca notícias no Google",
        tools=[google_search],
        instruction="""Você é um assistente de pesquisa. A sua tarefa é usar a ferramenta de buscas do google(google search) para recuperar as últimas
         notícias de lançamentos muito releva tes sobre o tópico abaixo. Foque em no maximo
        5 lançamentos relevantes, com base na quantidade e entusiasmo das notícias sobre ele.
        Se um tema tiver poucas notícias ou reações entusiasmadas, é possível que ela não seja tão relevante e pode ser substituido por outro que tenha mais. Esses lançamentos relevantes devem ser atuais,
         de no maximo um mês antes da data de hoje"""
    )
    entrada_do_agente_buscador = f"Tópico: {topico}\nDia de hoje: {dia_de_hoje}"
    # Executa o agente
    lancamentos_buscados = call_agent(buscador, entrada_do_agente_buscador)
    return lancamentos_buscados

In [ ]:
################################################
# --- Agente 2: Planejador de posts --- #
################################################
def agente_planejador(topico, lancamentos_buscados):
    planejador = Agent(
        name="agente_planejador",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente Planejador #################################################
        instruction=""" Você é um agente de planejamento de conteúdo, especialista em redes sociais. Com base na lista de lançamentos mais recentes e relevantes buscados, você deve:
        usar a ferramenta do Google (google_search) para criar um plano sobre quais são os pontos mais relevantes que poderíamos abordar em um post sobre cada um deles. você tambem pode usar o (google_search)
        para encontrar mais infromações sobre os temas e aprofundar.
        Ao final, você irá escolner o tema mais relevante entre eles com base nas suas pesquisas e retornar esse tema, seus pontos mais relevantes, e um plano com os assuntos a serem abordados
        no post que será escrito posteriormente
        """,
        description="Agente que planeja posts",
        tools=[google_search]
    )

    entrada_do_agente_planejador = f"Tópico:{topico}\nLançamentos buscados: {lancamentos_buscados}"
    # Executa o agente
    plano_do_post = call_agent(planejador, entrada_do_agente_planejador)
    return plano_do_post

In [ ]:
######################################
# --- Agente 3: Redator do Post --- #
######################################
def agente_redator(topico, plano_de_post):
    redator = Agent(
        name="agente_redator",
        model="gemini-2.0-flash",
        instruction="""
            Você é um Redator Criativo especializado em criar posts virais para redes sociais.
            Você escreve posts para a empresa Alura, a maior escola online de tecnologia do Brasil.
            Utilize o tema fornecido no plano de post e os pontos mais relevantes fornecidos e, com base nisso,
            escreva um rascunho de post para Instagram sobre o tema indicado.
            O post deve ser engajador, informativo, com linguagem simples e incluir 2 a 4 hashtags no final.
            """,
        description="Agente redator de posts engajadores para Instagram"
    )
    entrada_do_agente_redator = f"Tópico: {topico}\nPlano de post: {plano_de_post}"
    # Executa o agente
    rascunho = call_agent(redator, entrada_do_agente_redator)
    return rascunho

In [ ]:
##########################################
# --- Agente 4: Revisor de Qualidade --- #
##########################################
def agente_revisor(topico, rascunho_gerado):
    revisor = Agent(
        name="agente_revisor",
        model="gemini-2.0-flash",
        instruction="""
            Você é um Editor e Revisor de Conteúdo meticuloso, especializado em posts para redes sociais, com foco no Instagram.
            Por ter um público jovem, entre 18 e 30 anos, use um tom de escrita adequado. Use o (google_search) para verificar se as informações estão corretas
            Revise o rascunho de post de Instagram abaixo sobre o tópico indicado, verificando clareza, concisão, correção e tom.
            Se o rascunho estiver bom, responda apenas 'O rascunho está ótimo e pronto para publicar!'.
            Caso haja problemas, aponte-os e sugira melhorias.
            """,
        description="Agente revisor de post para redes sociais.",
                tools=[google_search]
    )
    entrada_do_agente_revisor = f"Tópico: {topico}\nRascunho: {rascunho_gerado}"
    # Executa o agente
    texto_revisado = call_agent(revisor, entrada_do_agente_revisor)
    return texto_revisado

In [ ]:
data_de_hoje = date.today().strftime("%d/%m/%Y")

print("🚀 Iniciando o Sistema de Criação de Posts para Instagram com 4 Agentes 🚀")

# --- Obter o Tópico do Usuário ---
topico = input("❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: ")
if  not topico:
  print("voce esqueceu de digitar o tópico")
else:
  print(f"O tópico escolhido foi: {topico}")

# Inserir lógica do sistema de agentes ################################################

#Agente Buscador em ação###################################
lancamentos_buscados = agente_buscador(topico, data_de_hoje)

print("/n--- Resultado do agente buscador ---\n")
display(to_markdown(lancamentos_buscados))

#Agente planejador em ação #################################################

print("---------------------------------------")

lancamentos_planejados = agente_planejador(topico, lancamentos_buscados)

print("\n--- Resultado do agente planejador ---\n")
display(to_markdown(lancamentos_planejados))

print("---------------------------------------")

#Agente redator em ação #################################
rascunho = agente_redator(topico, lancamentos_planejados)

print("\n--- Resultado do agente redator ---\n")
display(to_markdown(rascunho))
print("---------------------------------------")

#Agente revisor em ação #######
revisao = agente_revisor(topico, rascunho)

print("\n--- Resultado do agente revisor ---\n")
display(to_markdown(revisao))
print("---------------------------------------")

🚀 Iniciando o Sistema de Criação de Posts para Instagram com 4 Agentes 🚀
❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: Agentes de IA
O tópico escolhido foi: Agentes de IA
/n--- Resultado do agente buscador ---



> Para encontrar os lançamentos mais relevantes sobre agentes de IA no último mês, vou usar o Google Search para identificar as notícias e anúncios mais comentados e relevantes.
> 
> 
> Com base nas notícias e artigos mais recentes sobre agentes de IA, aqui estão alguns dos lançamentos e avanços mais relevantes no último mês (Abril-Maio de 2025):
> 
> 1.  **IBM Watsonx Orchestrate:** A IBM anunciou atualizações para o Watsonx Orchestrate, oferecendo soluções para integrar, customizar e implementar agentes de IA. Inclui agentes pré-construídos para RH, vendas e compras, integrados com mais de 80 aplicações empresariais (Adobe, AWS, Microsoft, Oracle, Salesforce, SAP, ServiceNow, Workday). Também lançou o Agent Catalog, que simplifica o acesso a esses agentes.
> 2.  **Aquisição da Windsurf pela OpenAI:** A OpenAI adquiriu a Windsurf (anteriormente Codeium) por US$ 3 bilhões. A Windsurf é um ambiente de desenvolvimento colaborativo que integra desenvolvedores e agentes inteligentes, permitindo codificação interativa e colaboração em tempo real.
> 3.  **Microsoft 365 Copilot Wave 2 Spring:** A Microsoft lançou o Microsoft 365 Copilot Wave 2 Spring, que inclui os Agentes Researcher e Analyst, alimentados por modelos de raciocínio profundo da OpenAI.
> 4.  **Avanços em Modelos de Linguagem:** Empresas como OpenAI (GPT-4), Anthropic (Claude) e Mistral AI estão aprimorando seus modelos de linguagem, que são a base para o raciocínio dos agentes de IA. Modelos open-source como o Llama 3 também estão sendo utilizados.
> 5.  **Novas Ferramentas e Plataformas de Desenvolvimento:** Espera-se que novos frameworks e plataformas de desenvolvimento de agentes reduzam a barreira de entrada, fornecendo protocolos de comunicação padrão, políticas de governança e gerenciamento de ciclo de vida.
> 
> Esses lançamentos indicam um foco crescente na integração de agentes de IA em diversas áreas empresariais, com melhorias tanto nos modelos de linguagem quanto nas ferramentas para implementar e gerenciar esses agentes.
> 


---------------------------------------

--- Resultado do agente planejador ---



> Com base nos lançamentos e avanços recentes em agentes de IA, vou criar um plano detalhado para um post de mídia social sobre o tema, focando no lançamento mais relevante e com maior potencial de engajamento para o público.
> 
> Primeiro, vamos buscar mais informações sobre cada um dos lançamentos para identificar os pontos mais interessantes:
> 
> 
> Após analisar os dados coletados, o lançamento da **Microsoft 365 Copilot Wave 2 Spring** parece ser o tópico mais relevante para um post de mídia social, devido ao seu potencial de impactar diretamente a produtividade e o fluxo de trabalho de muitos usuários empresariais.
> 
> ### Tópico Escolhido: Microsoft 365 Copilot Wave 2 Spring
> 
> **Pontos Relevantes:**
> 
> *   **Novos Agentes Researcher e Analyst:** Detalhes sobre como esses agentes de raciocínio profundo da OpenAI podem auxiliar em pesquisas complexas e análise de dados, respectivamente.
> *   **Agent Store:** A facilidade de encontrar, fixar e usar diversos agentes, tanto da Microsoft quanto de terceiros (Jira, Monday.com, Miro).
> *   **Copilot Search:** A capacidade de encontrar informações rapidamente no ambiente de trabalho usando linguagem natural, com resumos instantâneos.
> *   **Copilot Studio:** A plataforma para criar e personalizar agentes de IA para necessidades empresariais únicas.
> *   **Controles de TI:** Ferramentas para gerenciar, governar e medir o impacto dos agentes na organização.
> 
> **Plano de Conteúdo para o Post:**
> 
> 1.  **Título:** "Microsoft 365 Copilot Wave 2: A Revolução dos Agentes de IA no Seu Trabalho"
> 2.  **Introdução (2-3 linhas):**
> 
>     *   Mencionar que a Microsoft lançou a Wave 2 do Copilot, focando em colaboração humano-agente.
>     *   Destacar que esta atualização redefine a produtividade e a tomada de decisões nas empresas.
> 3.  **Destaque dos Agentes Researcher e Analyst (3-4 linhas):**
> 
>     *   Explicar como o Researcher ajuda a realizar pesquisas complexas com alta precisão.
>     *   Descrever como o Analyst transforma dados brutos em insights acionáveis rapidamente.
>     *   Mencionar a disponibilidade desses agentes através do Agent Store.
> 4.  **Exploração do Agent Store (2-3 linhas):**
> 
>     *   Enfatizar a facilidade de acesso a agentes da Microsoft e de parceiros como Jira, Monday.com, Miro.
>     *   Incentivar os usuários a explorar e personalizar seus fluxos de trabalho com esses agentes.
> 5.  **Copilot Search e Copilot Studio (3-4 linhas):**
> 
>     *   Explicar como o Copilot Search facilita a busca de informações no ambiente de trabalho com linguagem natural.
>     *   Destacar a capacidade do Copilot Studio de criar agentes personalizados para necessidades específicas da empresa.
> 6.  **Controles de TI e Segurança (2-3 linhas):**
> 
>     *   Mencionar as ferramentas de gerenciamento e governança de agentes no Copilot Control System.
>     *   Destacar a importância da segurança dos dados com as políticas do Microsoft Purview.
> 7.  **Chamada para Ação (1 linha):**
> 
>     *   Incentivar os usuários a explorar as novas funcionalidades e a compartilhar suas experiências.
> 
> 8.  **Hashtags:**
> 
>     *   \#Microsoft365 \#Copilot \#IA \#Produtividade \#AgentesDeIA \#OpenAI
> 
> **Assuntos a serem abordados no post:**
> 
> *   **O que é a Microsoft 365 Copilot Wave 2 Spring:**
> 
>     *   Uma atualização que visa revolucionar a forma como trabalhamos, integrando agentes de IA para aumentar a produtividade.
> *   **Principais Novidades:**
> 
>     *   **Agentes Researcher e Analyst:**
>         *   Como eles funcionam e como podem transformar a pesquisa e análise de dados.
>     *   **Agent Store:**
>         *   Acesso facilitado a diversos agentes, tanto da Microsoft quanto de terceiros.
>         *   Exemplos de agentes de parceiros (Jira, Monday.com, Miro) e como eles se integram ao fluxo de trabalho.
>     *   **Copilot Search:**
>         *   Busca inteligente e contextualizada no ambiente de trabalho.
>         *   Como encontrar arquivos, e-mails e conversas usando linguagem natural.
>     *   **Copilot Studio:**
>         *   Criação e personalização de agentes para necessidades específicas.
>         *   A importância de adaptar a IA às necessidades únicas de cada empresa.
>     *   **Controles de TI e Segurança:**
>         *   Ferramentas para gerenciar, governar e medir o impacto dos agentes.
>         *   Segurança dos dados e conformidade com políticas de proteção.
> *   **Benefícios Gerais:**
> 
>     *   Aumento da produtividade e eficiência.
>     *   Melhor tomada de decisões com insights precisos.
>     *   Automação de tarefas complexas.
> *   **Público-Alvo:**
> 
>     *   Profissionais que usam o Microsoft 365.
>     *   Líderes de TI responsáveis pela implementação de novas tecnologias.
>     *   Interessados em IA e automação no ambiente de trabalho.
> 


---------------------------------------

--- Resultado do agente redator ---



> ## Rascunho de Post para Instagram:
> 
> **Título:** Microsoft 365 Copilot Wave 2: A Revolução dos Agentes de IA no Seu Trabalho 🚀
> 
> Prepare-se para uma transformação radical na sua forma de trabalhar! A Microsoft acaba de lançar a Wave 2 do Copilot, uma atualização focada na colaboração entre humanos e agentes de IA, elevando a produtividade e a tomada de decisões a um novo patamar. 🤯
> 
> Já imaginou ter um agente que realiza pesquisas complexas com precisão cirúrgica? 🔎 O **Researcher** faz isso! E que tal um especialista em transformar dados brutos em insights valiosos em segundos? 📊 Conheça o **Analyst**! Ambos estão disponíveis no Agent Store, prontos para turbinar seu dia a dia.
> 
> No **Agent Store**, você encontra uma variedade incrível de agentes da Microsoft e de parceiros como Jira, Monday.com e Miro. 🤩 Explore, personalize e adapte os agentes ao seu fluxo de trabalho!
> 
> Precisa encontrar algo no seu ambiente de trabalho? O **Copilot Search** te ajuda a encontrar arquivos, e-mails e conversas usando linguagem natural. 🗣️ E com o **Copilot Studio**, você pode criar agentes personalizados para as necessidades específicas da sua empresa. 🏢
> 
> E a segurança? Fique tranquilo! O Copilot Control System oferece ferramentas robustas para gerenciar e governar os agentes, garantindo a proteção dos seus dados com as políticas do Microsoft Purview. 🔒
> 
> Pronto para experimentar o futuro do trabalho? ✨ Explore as novas funcionalidades, personalize seus agentes e compartilhe suas experiências!
> 
> \#Microsoft365 \#Copilot \#IA \#Produtividade \#AgentesDeIA
> 


---------------------------------------

--- Resultado do agente revisor ---



> O rascunho está bom, mas podemos deixá-lo ainda mais atraente para o público do Instagram. Aqui estão algumas sugestões:
> 
> *   **Título:** O título está bom, mas pode ser mais direto e chamativo. Que tal algo como: "Copilot Wave 2: A IA que vai EXPLODIR sua Produtividade!" ou "Chegou a Revolução da IA no Trabalho: Microsoft Copilot Wave 2!"
> 
> *   **Emojis:** Use mais emojis ao longo do texto para quebrar a monotonia e deixar o post mais visualmente interessante. Emojis relevantes podem ajudar a destacar os benefícios e funcionalidades.
> 
> *   **Linguagem:** Ajuste a linguagem para ser ainda mais informal e direta. Use gírias e expressões que o público jovem usa para criar uma conexão maior. Por exemplo, em vez de "precisão cirúrgica", você poderia dizer "com uma precisão absurda".
> 
> *   **Call to Action:** Incentive mais a interação. No final, adicione uma pergunta como: "Qual agente de IA você está mais ansioso para testar?" ou "Já usou o Copilot? Conta pra gente sua experiência!".
> 
> *   **Hashtags:** As hashtags estão boas, mas adicione algumas mais específicas e populares para aumentar o alcance, como #InteligenciaArtificial, #Tech, #Inovacao, #FuturoDoTrabalho.
> 
> **Sugestão de Rascunho Revisado:**
> 
> **Título:** Copilot Wave 2: A IA que vai EXPLODIR sua Produtividade! 🚀🤯
> 
> Se prepare pra dar um UPGRADE GIGANTE na sua vida profissional! A Microsoft lançou a Wave 2 do Copilot, e a parada é SÉRIA: IA trabalhando junto com você pra turbinar sua produtividade e te ajudar a tomar decisões INCRÍVEIS. 🤩
> 
> Já pensou em ter um agente que faz pesquisas COMPLEXAS rapidinho? 🔎 O **Researcher** faz isso! E um especialista em transformar aqueles dados chatos em insights que VALEM OURO? 📊 É o **Analyst**, e os dois estão te esperando no Agent Store pra dar um gás no seu dia a dia. 🔥
> 
> No **Agent Store**, tem um MONTÃO de agentes da Microsoft e de parceiros como Jira, Monday.com e Miro. 🤩 É só escolher, personalizar e mandar ver!
> 
> Precisa achar AQUELA informação no meio do caos? O **Copilot Search** acha arquivos, e-mails e conversas usando a sua LÍNGUA! 🗣️ E com o **Copilot Studio**, você CRIA seus próprios agentes pra resolver os BOs da sua empresa. 🏢
> 
> E a segurança? Relaxa! O Copilot Control System protege seus dados com as políticas do Microsoft Purview. 🔒
> 
> Bora pro FUTURO? ✨ Teste as novidades, personalize seus agentes e conta pra gente: qual agente você tá mais ansioso pra usar? 👇
> 
> \#Microsoft365 \#Copilot \#IA \#Produtividade \#AgentesDeIA #InteligenciaArtificial #Tech #Inovacao #FuturoDoTrabalho
> 
> Antes de publicar, use o Google para confirmar se as informações sobre o Copilot Wave 2 e os agentes (Researcher, Analyst, Copilot Search, Copilot Studio) estão corretas e atualizadas.
> 
> Com base nas informações mais recentes, o rascunho revisado está ainda melhor e pronto para ser publicado após as seguintes alterações:
> 
> *   Confirmei que a Microsoft lançou a Wave 2 do Copilot com os agentes Researcher e Analyst.
> *   O Copilot Search realmente existe e busca simplificar a pesquisa, fornecendo respostas resumidas e com fontes citadas.
> *   O Copilot Studio permite criar agentes personalizados, integrando dados da empresa e automatizando tarefas.
> *   O Microsoft Purview garante a segurança e governança dos dados.
> 
> **Rascunho Final:**
> 
> **Título:** Copilot Wave 2: A IA que vai EXPLODIR sua Produtividade! 🚀🤯
> 
> Se prepare pra dar um UPGRADE GIGANTE na sua vida profissional! A Microsoft lançou a Wave 2 do Copilot, e a parada é SÉRIA: IA trabalhando junto com você pra turbinar sua produtividade e te ajudar a tomar decisões INCRÍVEIS. 🤩
> 
> Já pensou em ter um agente que faz pesquisas COMPLEXAS rapidinho? 🔎 O **Researcher** faz isso! E um especialista em transformar aqueles dados chatos em insights que VALEM OURO? 📊 É o **Analyst**, e os dois estão te esperando no Agent Store pra dar um gás no seu dia a dia. 🔥
> 
> No **Agent Store**, tem um MONTÃO de agentes da Microsoft e de parceiros como Jira, Monday.com e Miro. 🤩 É só escolher, personalizar e mandar ver!
> 
> Precisa achar AQUELA informação no meio do caos? O **Copilot Search** acha arquivos, e-mails e conversas usando a sua LÍNGUA! 🗣️ E com o **Copilot Studio**, você CRIA seus próprios agentes pra resolver os BOs da sua empresa. 🏢
> 
> E a segurança? Relaxa! O Copilot Control System protege seus dados com as políticas do Microsoft Purview. 🔒
> 
> Bora pro FUTURO? ✨ Teste as novidades, personalize seus agentes e conta pra gente: qual agente você tá mais ansioso pra usar? 👇
> 
> \#Microsoft365 \#Copilot \#IA \#Produtividade \#AgentesDeIA #InteligenciaArtificial #Tech #Inovacao #FuturoDoTrabalho
> 


---------------------------------------
